# Recap

In [ ]:
from crewai.flow.flow import Flow, start

class SimpleFlow(Flow):

    @start()
    def initialize(self):
        print("Flow started")

flow = SimpleFlow()
await flow.kickoff_async()

In [ ]:
from crewai.flow.flow import Flow, listen, start

class SequentialFlow(Flow):

    @start()
    def first_task(self):
        print("Step 1: Fetching data")
        return "data_fetched"

    @listen(first_task)
    def second_task(self, result):
        print(f"Step 2: Processing {result}")

flow = SequentialFlow()
await flow.kickoff_async()

In [ ]:
from crewai.flow.flow import Flow, listen, or_, start

class OrFlow(Flow):

    @start()
    def fetch_from_api(self):
        return "API data"

    @start()
    def read_from_db(self):
        return "Database record"

    @listen(or_(fetch_from_api, read_from_db))
    def process_data(self, result):
        print(f"Processing: {result}")

flow = OrFlow()
await flow.kickoff_async()

In [ ]:
from crewai.flow.flow import Flow, listen, and_, start

class AndFlow(Flow):

    @start()
    def step_one(self):
        print("Step 1: Collecting user input")
        return "User data"

    @start()
    def step_two(self):
        print("Step 2: Validating input")
        return "Validated data"

    @listen(and_(step_one, step_two))
    def final_step(self):
        print("All conditions met. Proceeding with final step.")

flow = AndFlow()
await flow.kickoff_async()

In [ ]:
import random
from crewai.flow.flow import Flow, listen, router, start

class RouterFlow(Flow):

    @start()
    def classify_request(self):
        request_type = random.choice(["urgent", "normal"])
        print(f"Request classified as: {request_type}")
        return request_type

    @router(classify_request)
    def handle_request(self, classification):
        return "handle_urgent" if classification == "urgent" else "handle_normal"

    @listen("handle_urgent")
    def urgent_handler(self):
        print("Handling urgent request")

    @listen("handle_normal")
    def normal_handler(self):
        print("Handling normal request")

flow = RouterFlow()
await flow.kickoff_async()


In [ ]:
from crewai.flow.flow import Flow, listen, start

class StateFlow(Flow):

    @start()
    def initialize_state(self):
        self.state["count"] = 1
        print(f"Initial count: {self.state['count']}")

    @listen(initialize_state)
    def increment_count(self):
        self.state["count"] += 1
        print(f"Updated count: {self.state['count']}")

flow = StateFlow()
await flow.kickoff_async()


In [ ]:
from crewai.flow.flow import Flow, listen, start
from pydantic import BaseModel

# Defining a structured state model
class CounterState(BaseModel):
    count: int = 0

class StructuredStateFlow(Flow[CounterState]):

    @start()
    def initialize_state(self):
        print(f"Initial count: {self.state.count}")
        self.state.count = 1

    @listen(initialize_state)
    def increment_count(self):
        self.state.count += 1
        print(f"Updated count: {self.state.count}")

flow = StructuredStateFlow()
await flow.kickoff_async()
